In [1]:
import os;
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import PIL
from timeit import default_timer as timer

In [2]:
class FeatureExtractor():
    def __init__(self):
        self.model = models.vgg16(pretrained=True);
        self.layer = self.model.classifier[-3];
        self.device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device('cpu');
        print(self.device)
        self.model.to(self.device)
        self.model.eval();
        self.resizer = transforms.Resize((224,224));
        self.transformer = transforms.ToTensor();
    
    
    def get(self, img):
        features = torch.zeros(len(img), 4096);
        def copy_output(model, inp, out):
            features.copy_(out.data)
        
        img = [ self.transformer(self.resizer(i)) for i in img ];
        images = torch.stack(img).to( self.device );
                    
        
        self.layer.register_forward_hook(copy_output);
        self.model(images);
        
        return features.numpy();
        
extractor = FeatureExtractor();


cuda:0


In [7]:
start = timer()
directory = 'dataset/train/PNEUMONIA'
i = 0
j = 0
train_PNEUMONIA = np.zeros((3870,4096))
for fname in os.listdir(directory):
    if i==0:
        images = []
    img = Image.open(directory + "/" + fname).convert('RGB')
    
    if i < 10:
        images.append(img)
        i += 1
    
    if i == 10:
        tmp = extractor.get(images);
        train_PNEUMONIA[10*j : 10*(j+1) , :] = tmp
        j += 1 
        i = 0
    
print(train_PNEUMONIA.shape)
print("It took {p:.2f} seconds".format(p = timer()-start))

(3870, 4096)
It took 706.48 seconds


In [9]:
start = timer()
directory = 'dataset/train/NORMAL'
i = 0
j = 0
train_NORMAL = np.zeros((1340,4096))
for fname in os.listdir(directory):
    if i==0:
        images = []
    img = Image.open(directory + "/" + fname).convert('RGB')
    
    if i < 10:
        images.append(img)
        i += 1
    
    if i == 10:
        tmp = extractor.get(images);
        train_NORMAL[10*j : 10*(j+1) , :] = tmp
        j += 1 
        i = 0
    
print(train_NORMAL.shape)
print("It took {p:.2f} seconds".format(p = timer()-start))

(1340, 4096)
It took 265.30 seconds


In [10]:
X_train = np.concatenate((train_NORMAL, train_PNEUMONIA), axis=0)
print(X_train.shape)

(5210, 4096)


In [11]:
y_train = np.concatenate( ( np.zeros(train_NORMAL.shape[0]), np.ones(train_PNEUMONIA.shape[0]) ) , axis = 0 )
print(  (y_train == 0).sum() )

1340


In [12]:
start = timer()
directory = 'dataset/test/PNEUMONIA'
i = 0
j = 0
test_PNEUMONIA = np.zeros((390,4096))
for fname in os.listdir(directory):
    if i==0:
        images = []
    img = Image.open(directory + "/" + fname).convert('RGB')
    
    if i < 10:
        images.append(img)
        i += 1
    
    if i == 10:
        tmp = extractor.get(images);
        test_PNEUMONIA[10*j : 10*(j+1) , :] = tmp
        j += 1 
        i = 0
    
print(test_PNEUMONIA.shape)
print("It took {p:.2f} seconds".format(p = timer()-start))

(390, 4096)
It took 82.14 seconds


In [13]:
start = timer()
directory = 'dataset/test/NORMAL'
i = 0
j = 0
test_NORMAL = np.zeros((230,4096))
for fname in os.listdir(directory):
    if i==0:
        images = []
    img = Image.open(directory + "/" + fname).convert('RGB')
    
    if i < 10:
        images.append(img)
        i += 1
    
    if i == 10:
        tmp = extractor.get(images);
        test_NORMAL[10*j : 10*(j+1) , :] = tmp
        j += 1 
        i = 0
    
print(test_NORMAL.shape)
print("It took {p:.2f} seconds".format(p = timer()-start))

(230, 4096)
It took 53.88 seconds


In [14]:
X_test = np.concatenate((test_NORMAL, test_PNEUMONIA), axis=0)
print(X_test.shape)
y_test = np.concatenate( ( np.zeros(test_NORMAL.shape[0]), np.ones(test_PNEUMONIA.shape[0]) ) , axis = 0 )
print(  (y_test == 0).sum() )

(620, 4096)
230


In [29]:
np.save('X_train',X_train)
np.save('y_train',y_train)
np.save('X_test',X_test)
np.save('y_test',y_test)